# This notebook explaines the sector classification process

## Get the refined dictionary for the project description corpus
First, we use nltk package to split project description strings into single words, get the postag of each word, remove all the stop words and lemmatize the words and get a list of words, which is the first version of dictionary. 

csv file containing projection description for each doc: [pro_desc.csv]

This dictionary, denoted by D1, contains many tokens which are non English words, for example: rd, imed. So we use a dictionary consisting of English words, denoted by ED (shorthand for English Dictionary), to find which words D1 are also ED. But among those non-English words, there are many consisting of two English words, for example, suchevidence, fullywithdrawn. It takes much time to seperate all those composition words. So we choose to only seperate those words contributing to the classification of sectors.

We found the descriptions of different sectors from the world bank website: [World_bank_website](https://projects.worldbank.org/en/projects-operations/project-sector?lang=en&page=), and try to seperate words containing any word appears in those descriptions. After above preprocessing, we have a dictionaty of around 45000 words saved in a csv file together with its frequency, whether it is in ED, and its postag. 

python file to get the dictionary:[refined_dic_python_file]()
csv file for this dictionary:[dictionary_csv]()
dictionry which only contains words in ED which is noun or adj: [in_word_n_adj_csv]


## Use pretrained Glove word embeddings to get an embedding for each document
Here we use the Glove word embedding of dimenation 300 trained from Wikipedia, a link to the. We finally choose 300 dimension because it gives best clustering results among all dimensional vectors.

text file of pretrained Glove words embedding of 300 dim:[glove_300d]

The embedding vector for project description from a single document is defined as the average of the vetors of words in this projection description and the common dictionary. Here this common dictionary we use for doc embedding is not the one obtained above because of the vocabulary size. To cut down the dictionary size and reduce the noise, we only choose the nouns and adjectives appeared in both ED and the pretrained Glove words. Now we have a dictionary of around 6000 words. Then we compute the doc embedding as the average.

We also get an embedding vector for each sector using the description provided by world bank.


Python code to get the above dictionary: [glove.py]
The csv file of above dicionary together with words embeddings: [refined_common_dic_csv]
Python code to get the document embedding: [doc_embedding_py]
The csv file storing the doc embedding: [glove_coef.csv]
The csv file storing the sector description embedding: [refined_sec_glove_coef_csv]




## Use k-mean to get the clustering
We have tried the DBSCAN and K-means for the clusterting, and K_means performs well. Because we would like to have 11 sector clusters. It is easier for K-mean to controll the size of clusters. And we use TNSE for visualization.

We use TNSE to find the projection of these vectors to 2d plane, there is some cluster showing. In the picture below, the yellow points are each document and the other colored points are vectors representing sector description from world bank.

python code to perfom TNSE:[tnse.py]()
the 2dimensional vector obtained for TNSE: [refined_glove_sec_tnse.csv]
plotting code for tnse vectors: [plot_sec.py]




### Details of K-Means
We require the K-Means algorithm to have 11 clusters. If a doc is inside the same cluster as the sector description from world bank website, then the doc is classifed as in that sector.

The cluster result is slightly different for each run of K-Mean. So we run K-Mean 500 times to get the probability distribution of each doc in different sectors, and we choose the sector with highest probability as the sector for the documents.

python code to perform K-mean: [multi_k_mean_py]()
sector distribution for each doc: [freq_csv]()
assigne sector to each document: [new_accuracy_kmean.py]()
csv storing sector classification:[sector_classification]()


## final adjustment according to the sampling accuracy
The 11 sectors provided by world banks:
* Agriculture
* Education
* Energy & Extractives
* Finanial Sector
* Health
* Industry & Trader/Ser
* Infor & communication
* Public Admin
* Social Protection
* Transportation
* Water/Saint/Waste

Result analysis of K-mean: 
* The sector Health and Social Protection can not be distinguished. To be more precise, the sector descriptions for health and social protection are always in the same cluster.
* There are around documents which are not classified in any sector. If for each K-mean, there is some cluster which contains no sector descriptions, then the docs in this cluster are "not_classified". Because of the non-dinstinction of health and social protection, there are always some docs which are not classifed.
* Infor&Communication sector has most documents, but there are some which are misclassified.
